In [1]:
import numpy as np
import cv2
import time
import os
import pandas as pd
from tqdm import tqdm
from collections import deque
from random import shuffle

In [2]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d,avg_pool_2d, conv_3d, max_pool_3d, avg_pool_3d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.merge_ops import merge

def alexnet(width, height, lr, output=9):
    network = input_data(shape=[None, width, height, 1], name='input')
    network = conv_2d(network, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, output, activation='softmax')
    network = regression(network, optimizer='momentum',
                         loss='categorical_crossentropy',
                         learning_rate=lr, name='targets')

    model = tflearn.DNN(network,max_checkpoints=1, tensorboard_verbose=0, tensorboard_dir='log')

    return model

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
WIDTH = 160
HEIGHT = 120
LR = 1e-3
model = alexnet(WIDTH, HEIGHT, LR)

EPOCHS = 10
MODEL_NAME ='pygtaSA-car-fast-{}-{}-{}-epochs-120k-data.model'.format(LR, 'alexnet',EPOCHS)
model.load(MODEL_NAME)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
INFO:tensorflow:Restoring parameters from C:\Users\Ali\Desktop\collect\pygtaSA-car-fast-0.001-alexnet-10-epochs-120k-data.model


In [4]:
import random
import ctypes
import time
from ipynb.fs.full. directx_keypresses import PressKey,ReleaseKey,W,A,S,D
from PIL import ImageGrab
from ipynb.fs.full. getkeys import key_check

In [5]:
t_time = 0.09
def straight():
    if random.randrange(4) == 2:
        ReleaseKey(W)
    else:
        PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)
    print("straight")

def left():
    PressKey(W)
    PressKey(A)
    #ReleaseKey(W)
    ReleaseKey(D)
    #ReleaseKey(A)
    time.sleep(t_time)
    ReleaseKey(A)
    print("left")
    
def right():
    PressKey(W)
    PressKey(D)
    ReleaseKey(A)
    #ReleaseKey(W)
    #ReleaseKey(D)
    time.sleep(t_time)
    ReleaseKey(D)
    print("rigth")
    
def reverse():
    PressKey(S)
    ReleaseKey(A)
    ReleaseKey(W)
    ReleaseKey(D)
    print("reverse")

def forward_left():
    PressKey(W)
    PressKey(A)
    ReleaseKey(D)
    ReleaseKey(S)
    print("forward_left")
    
    
def forward_right():
    PressKey(W)
    PressKey(D)
    ReleaseKey(A)
    ReleaseKey(S)
    print("forward_right")
    
def reverse_left():
    PressKey(S)
    PressKey(A)
    ReleaseKey(W)
    ReleaseKey(D)
    print("reverse_left")
    
def reverse_right():
    PressKey(S)
    PressKey(D)
    ReleaseKey(W)
    ReleaseKey(A)
    print("reverse_right")

def no_keys():

    if random.randrange(0,3) == 1:
        PressKey(W)
    else:
        ReleaseKey(W)
    ReleaseKey(A)
    ReleaseKey(S)
    ReleaseKey(D)

In [7]:
def main():
    last_time = time.time()
    for i in list(range(4))[::-1]:
        print(i+1)
        time.sleep(1)

    paused = False
    while(True):
        
        if not paused:
       
            screen =np.array(ImageGrab.grab(bbox=(5,50,480,360)))
            print('loop took {} seconds'.format(time.time()-last_time))
            last_time = time.time()
            screen = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY)
            screen = cv2.resize(screen, (160,120))

            prediction = model.predict([screen.reshape(160,120,1)])
            print(prediction)
            max=np.argmax(prediction)
            print(max)
            
            if max==0:
                straight()
            elif max==1:
                reverse()
            elif max==2:
                left()
            elif max==3:
                right()
            elif max==4:
                forward_left()
            elif max==5:
                forward_right()
            elif max==6:
                reverse_left()
            elif max==7:
                reverse_right()
            elif max==8:
                no_keys()

        keys = key_check()

        # p pauses game and can get annoying.
        if 'T' in keys:
            if paused:
                paused = False
                time.sleep(1)
            else:
                paused = True
                ReleaseKey(A)
                ReleaseKey(W)
                ReleaseKey(D)
                time.sleep(1)

main()  

4
3
2
1
loop took 4.230671644210815 seconds
[[9.5271891e-01 3.6098126e-03 2.6894450e-02 1.5934838e-02 6.8013015e-04
  1.5962038e-04 1.0270065e-06 4.8972544e-07 8.2418626e-07]]
0
straight
loop took 1.50746750831604 seconds
[[9.5271891e-01 3.6098126e-03 2.6894450e-02 1.5934838e-02 6.8013015e-04
  1.5962038e-04 1.0270065e-06 4.8972544e-07 8.2418626e-07]]
0
straight
loop took 0.06248831748962402 seconds
[[9.5271891e-01 3.6098126e-03 2.6894450e-02 1.5934838e-02 6.8013015e-04
  1.5962038e-04 1.0270065e-06 4.8972544e-07 8.2418626e-07]]
0
straight
loop took 0.04880356788635254 seconds
[[9.5271891e-01 3.6098126e-03 2.6894450e-02 1.5934838e-02 6.8013015e-04
  1.5962038e-04 1.0270065e-06 4.8972544e-07 8.2418626e-07]]
0
straight
loop took 0.05097627639770508 seconds
[[9.5271891e-01 3.6098126e-03 2.6894450e-02 1.5934838e-02 6.8013015e-04
  1.5962038e-04 1.0270065e-06 4.8972544e-07 8.2418626e-07]]
0
straight
loop took 0.040630340576171875 seconds
[[9.5271891e-01 3.6098126e-03 2.6894450e-02 1.5934838

straight
loop took 0.050339460372924805 seconds
[[9.5271891e-01 3.6098126e-03 2.6894450e-02 1.5934838e-02 6.8013015e-04
  1.5962038e-04 1.0270065e-06 4.8972544e-07 8.2418626e-07]]
0
straight
loop took 0.040329933166503906 seconds
[[9.5271891e-01 3.6098126e-03 2.6894450e-02 1.5934838e-02 6.8013015e-04
  1.5962038e-04 1.0270065e-06 4.8972544e-07 8.2418626e-07]]
0
straight
loop took 0.062485694885253906 seconds
[[9.5271891e-01 3.6098126e-03 2.6894450e-02 1.5934838e-02 6.8013015e-04
  1.5962038e-04 1.0270065e-06 4.8972544e-07 8.2418626e-07]]
0
straight
loop took 0.04935407638549805 seconds
[[9.5271891e-01 3.6098126e-03 2.6894450e-02 1.5934838e-02 6.8013015e-04
  1.5962038e-04 1.0270065e-06 4.8972544e-07 8.2418626e-07]]
0
straight
loop took 0.05382370948791504 seconds
[[9.5271891e-01 3.6098126e-03 2.6894450e-02 1.5934838e-02 6.8013015e-04
  1.5962038e-04 1.0270065e-06 4.8972544e-07 8.2418626e-07]]
0
straight
loop took 0.06252861022949219 seconds
[[9.5271891e-01 3.6098126e-03 2.6894450e-02 1

KeyboardInterrupt: 